This notebook calculates the change metrics. Splitting change in effective,defective and ineffective

In [ ]:
import pickle
import torch
import pandas as pd

In [2]:
def load_data(model,mode, dataset, split):
#load predicted and gold bounding boxes

    try:

        #the predicted bounding box
        with open(r"/home/users/fschreiber/project/bboxes_"+model+"/"+dataset+"/"+split+"_pred_bbox_list.p","rb") as f:
            pred_bbox_list=list(pickle.load(f))

        if mode=="non_inc":
            #the target bounding box
            with open(r"/home/users/fschreiber/project/bboxes_noninc_"+model+"/"+dataset+"/"+split+"_pred_bbox_list.p","rb") as f:
                target_bbox_list=list(pickle.load(f))

        elif mode == "inc":
            #the target bounding box
            with open(r"/home/users/fschreiber/project/bboxes_"+model+"/"+dataset+"/"+split+"_target_bbox_list.p","rb") as f:
                target_bbox_list=list(pickle.load(f))
        else:
            print("The mode can only be non_inc or inc")
            return -1,-1,-1,-1,-1

        #the number of one sentence split up incrementally ("the left zebra" would have length 3)
        with open(r"/home/users/fschreiber/project/incremental_pickles/length_incremental_units/"+dataset+"_"+split+"_length_unit.p","rb") as f:
            inc_len=pickle.load(f)

        #the original model data split up incrementally
        data_model=torch.load("/home/users/fschreiber/project/ready_inc_data/"+dataset+"/"+dataset+"_"+split+".pth")

        with open(r"/home/users/fschreiber/project/binary_grouped/"+model+"/"+mode+"/"+dataset+split+".p","rb") as f:
            binary_grouped=pickle.load(f)

        
        if mode=="non_inc":
            target_bbox_list=[x for x,y in zip(target_bbox_list,inc_len) for _ in range(y)]
            
        if model=="TVG":
            pred_bbox_list,target_bbox_list=TVG_prep(pred_bbox_list,target_bbox_list)
                    

        return pred_bbox_list,target_bbox_list,inc_len,data_model,binary_grouped
    
    except FileNotFoundError as e:
        #print(e)
        
        return  -1,-1,-1,-1,-1
    


#TVG needs some extra adjustments to fit the same data format as Resc
def TVG_prep(pred_bbox_list,target_bbox_list):
    #print("TVG")
    for ind,(pred,targ) in enumerate (zip (pred_bbox_list,target_bbox_list)):

        pred=pred.view(1,-1)

        pred=xywh2xyxy(pred)
        pred=torch.clamp(pred,0,1)

        pred_bbox_list[ind]=pred

        targ=targ.view(1,-1)
        targ=xywh2xyxy(targ)

        target_bbox_list[ind]=targ
    return pred_bbox_list,target_bbox_list

#copied from TransVG needed to transform the bounding box vectors
def xywh2xyxy(x):
    x_c, y_c, w, h = x.unbind(-1)
    b = [(x_c - 0.5 * w), (y_c - 0.5 * h),
         (x_c + 0.5 * w), (y_c + 0.5 * h)]
    return torch.stack(b, dim=-1)


In [3]:
#copied from ReSC
def bbox_iou(box1, box2, x1y1x2y2=True):
    """
    Returns the IoU of two bounding boxes
    """
    if x1y1x2y2:
        # Get the coordinates of bounding boxes
        b1_x1, b1_y1, b1_x2, b1_y2 = box1[:, 0], box1[:, 1], box1[:, 2], box1[:, 3]
        b2_x1, b2_y1, b2_x2, b2_y2 = box2[:, 0], box2[:, 1], box2[:, 2], box2[:, 3]
    else:
        # Transform from center and width to exact coordinates
        b1_x1, b1_x2 = box1[:, 0] - box1[:, 2] / 2, box1[:, 0] + box1[:, 2] / 2
        b1_y1, b1_y2 = box1[:, 1] - box1[:, 3] / 2, box1[:, 1] + box1[:, 3] / 2
        b2_x1, b2_x2 = box2[:, 0] - box2[:, 2] / 2, box2[:, 0] + box2[:, 2] / 2
        b2_y1, b2_y2 = box2[:, 1] - box2[:, 3] / 2, box2[:, 1] + box2[:, 3] / 2

    # get the coordinates of the intersection rectangle
    inter_rect_x1 = torch.max(b1_x1, b2_x1)

    inter_rect_y1 = torch.max(b1_y1, b2_y1)

    inter_rect_x2 = torch.min(b1_x2, b2_x2)

    inter_rect_y2 = torch.min(b1_y2, b2_y2)

    
    # Intersection area
    inter_area = torch.clamp(inter_rect_x2 - inter_rect_x1, 0) * torch.clamp(inter_rect_y2 - inter_rect_y1, 0)
    # Union Area
    b1_area = (b1_x2 - b1_x1) * (b1_y2 - b1_y1)
    b2_area = (b2_x2 - b2_x1) * (b2_y2 - b2_y1)

    return inter_area / (b1_area + b2_area - inter_area + 1e-16)
    
#copied from TransVG needed to transform the bounding box vectors
def xywh2xyxy(x):
    x_c, y_c, w, h = x.unbind(-1)
    b = [(x_c - 0.5 * w), (y_c - 0.5 * h),
         (x_c + 0.5 * w), (y_c + 0.5 * h)]
    return torch.stack(b, dim=-1)


In [4]:
pred_bbox_list,target_bbox_list,inc_len,model,binary_grouped=load_data("ReSc","inc","gref","val")


In [5]:
#group sentences that belong to one incremental unit
def group_by_increment(bbox_list,inc_len):
    counter=0
    group_list=[]
    for i in inc_len:
        
        group_list.append(bbox_list[counter:counter+i])
        counter=counter+i
    return group_list

pred_group=group_by_increment(pred_bbox_list,inc_len)

In [8]:
def effectivness(bin, pred):
    # Initialize counters for different categories
    effective = 0
    ineffective = 0
    defective = 0
    
    # Initialize previous values for bin and pred
    prev = None
    pred_prev = None

    # Iterate through the elements of bin and pred in parallel
    for value, p in zip(bin, pred):
        
        #initalize the first value
        if prev is None:
            pred_prev = p
            if value == 1:
                effective += 1
            elif value == 0:
                defective += 1
            else: 
                print("Something is very wrong. The value given is:", value, "it should be 0 or 1.")
                break

            prev = value
            continue

        # Compare bin values to find change 
        if prev == 0 and value == 1:
            effective += 1
        elif prev == 0 and value == 0:
            # Check the intersection over union (IOU) between previous and current predictions
            if bbox_iou(pred_prev, p) < 0.5:
                ineffective += 1
        elif prev == 1 and value == 0:
            defective += 1
        
        # Update values 
        prev = value
        pred_prev = p

    return effective, ineffective, defective



bin = binary_grouped[5]
pred = pred_group[5]


effective, ineffective, defective = effectivness(bin, pred)

# Calculate and print the percentage of each category
print("Effective Change:", (effective) / (effective + defective + ineffective))
print("Defective Change:", (defective) / (effective + defective + ineffective))
print("Ineffective Change:", (ineffective) / (effective + defective + ineffective))
print("Total Change:", (effective + defective + ineffective) / len(bin))


Effective Change: 0.25
Defective Change: 0.5
Ineffective Change: 0.25
Total Change: 0.6666666666666666


In [9]:
# Initialize empty lists 
found_sets_list, effective_list, defective_list, ineffective_list, total_list = ([] for i in range(5))

# Define variables
mode = "inc"
model_input = "ReSc"
split_list = ["testB", "testA", "val", "test"]
dataset_list = ["unc", "unc+", "gref_umd", "gref"]

#load each split and dataset
for file in dataset_list:
   
    for split in split_list:
      
        pred_bbox_list, target_bbox_list, inc_len, model, binary_grouped = load_data(model_input, mode, file, split)
        
        if pred_bbox_list == -1 or target_bbox_list == -1 or inc_len == -1:
            pass
       
        else:
           
            print(file + split)
            found_sets_list.append(file + " " + split)

            # Initialize lists to store change
            effec = []
            defec = []
            ineffec = []
            total = []

            # Iterate through 'binary_grouped' and 'pred_bbox_list'
            for bin, pred in zip(binary_grouped, pred_bbox_list):
                
                # Calculate effectiveness, defectiveness, and ineffectiveness
                effective, ineffective, defective = effectivness(bin, pred)
                effec.append(effective)
                defec.append(defective)
                ineffec.append(ineffective)
                total.append((effective + defective + ineffective) / len(bin))

            #Take average
            effective_list.append((sum(effec)) / (sum(effec) + sum(defec) + sum(ineffec)))
            defective_list.append((sum(defec)) / (sum(effec) + sum(defec) + sum(ineffec)))
            ineffective_list.append((sum(ineffec)) / (sum(effec) + sum(defec) + sum(ineffec)))
            total_list.append(sum(total) / len(total))

# Create dictonary
data = {}

data["Dataset" + model_input + " " + mode] = found_sets_list
data["Effective"] = effective_list
data["Defective"] = defective_list
data["Ineffective"] = ineffective_list
data["Total Change"] = total_list

# Create Dataframe
df = pd.DataFrame(data)
df.set_index("Dataset" + model_input + " " + mode, inplace=True)
df = round(df, 2)

df


unctestB
unctestA
uncval
unc+testB
unc+testA
unc+val
gref_umdval
gref_umdtest
grefval


,Effective,Defective,Ineffective,Total Change
DatasetReSc inc,,,,
unc testB,0.48,0.35,0.17,0.53
unc testA,0.54,0.32,0.14,0.58
unc val,0.51,0.33,0.15,0.54
unc+ testB,0.41,0.40,0.20,0.54
unc+ testA,0.49,0.36,0.16,0.60
unc+ val,0.46,0.38,0.17,0.57
gref_umd val,0.40,0.40,0.20,0.32
gref_umd test,0.40,0.40,0.20,0.33
gref val,0.39,0.42,0.19,0.33
